In [ ]:
import xarray as xr
from matplotlib import pyplot as plt
import cftime._cftime as cft
import holoviews as hv

hv.extension('bokeh')

In [ ]:
def clean_coords(xrdataset, start_year):
    try:
        shift = cft.timedelta(days=360 * start_year)
        datetimeindex = (xrdataset.indexes['time_counter'] + shift).to_datetimeindex
    except:
        datetimeindex = xrdataset.indexes['time_counter']
    try:
        latitude  = xrdataset.nav_lat[:,0].values
        longitude = xrdataset.nav_lon[0,:].values
    except:
        latitude  = xrdataset.nav_lat_grid_T[:,0].values
        longitude = xrdataset.nav_lon_grid_T[0,:].values
    try:
        xrdataset = xrdataset.rename_dims({'x_grid_T': 'x' ,'y_grid_T': 'y'})
    except:
        pass
    try:
        xrdataset = xrdataset.assign_coords({'z': 5. - xrdataset.e3t.cumsum('deptht')})
    except:
        pass
    try:
        xrdataset = xrdataset.assign_coords({'z': 5. - xrdataset.e3u.cumsum('depthu')})
    except:
        pass
    try:
        xrdataset = xrdataset.assign_coords({'z': 5. - xrdataset.e3v.cumsum('depthv')})
    except:
        pass
    try:
        xrdataset = xrdataset.assign_coords({'z': 5. - xrdataset.e3w.cumsum('depthw')})
    except:
        pass
    return xrdataset.assign_coords(dict({'time_counter':datetimeindex, 'y': latitude, 'x': longitude}))

In [ ]:
def read_abort(path, dict_vars={'votemper': 'T', 'vosaline': 'S', 'vozocrtx': 'U', 'vomecrty': 'V', 'vovecrtz': 'W', 'sossheig': 'H'}):
    abort = xr.load_dataset(path)
    abort = abort.rename(dict_vars)
    abort = abort.set_coords(['nav_lon', 'nav_lat'])
    return(abort[list(dict_vars.values())])

In [ ]:
%%output holomap='scrubber'
%%opts QuadMesh style(cmap='RdBu_r') plot[colorbar=True] 
%%opts QuadMesh [width=500, height=400]
hv_ds = hv.Dataset(U.uoce.isel(depthu=-4))
hv_ds.to(hv.QuadMesh, ['x', 'y'])